In [1]:
import pandas as pd
import time, requests, json, random, os
#from selenium import webdriver
from splinter import Browser
from bs4 import BeautifulSoup as bs
from subprocess import call

In [2]:
#*   Prohibits truncatation of data in prints.
pd.set_option('display.max_colwidth', -1)

In [3]:
# All Academy Awards winners .csv loaded as a DataFrame 
# (SOURCE: https://datahub.io/rufuspollock/oscars-nominees-and-winners)
allWinAndNom = pd.read_csv("/Users/nicolespaar/Desktop/oscars_winners.csv")

def getWinnersDF_Film(allWinAndNOm):
    allWinners_year = []
    allWinners_category = []
    allWinners_entity = []
    
    # Rows are iterrated and those that have a True bool in the `winner` attribute are
    # appended to the above declared holding arrays.
    for row in allWinAndNom.iterrows():
        if row[1]['winner'] == True:
            allWinners_year.append(row[1]['year'])
            allWinners_category.append(row[1]['category'])
            allWinners_entity.append(row[1]['entity'])
    
    # A new $winners_data dict is created from the above iterrator's resulting arrays. Then a
    # DataFrame is created from the dict.
    winners_data = {'Year': allWinners_year, 'Category': allWinners_category, 'Entity': allWinners_entity}
    winnersDF = pd.DataFrame(data=winners_data)
    
    # The name of the current `Best Picture` Academy Award was called several things
    # throughout the years. Each variation is extracted as a new DataFrame.
    winnersDF_films_A = winnersDF[winnersDF['Category'] == 'BEST PICTURE'].copy()
    winnersDF_films_B = winnersDF[winnersDF['Category'] == 'OUTSTANDING PRODUCTION'].copy()
    winnersDF_films_C = winnersDF[winnersDF['Category'] == 'OUTSTANDING MOTION PICTURE'].copy()
    winnersDF_films_D = winnersDF[winnersDF['Category'] == 'BEST MOTION PICTURE'].copy()
    winnersDF_films_E = winnersDF[winnersDF['Category'] == 'OUTSTANDING PICTURE'].copy()
    
    winners_years = []
    winners_entities = []
    
    # The first two years have incorrect `Entity` values so they are manually altered.
    for row in winnersDF_films_E.iterrows():
        winners_years.append(row[1]['Year'])
    winners_entities.append("wings")
    winners_entities.append("the broadway melody")

    # Each of the other DataFrames holding best picture winners by varying category name are
    # iterrated and added to the above winners arrays. The entity is also changed to lowercase.
    for row in winnersDF_films_B.iterrows():
        winners_years.append(row[1]['Year'])
        winners_entities.append(row[1]['Entity'].lower())
    for row in winnersDF_films_C.iterrows():
        winners_years.append(row[1]['Year'])
        winners_entities.append(row[1]['Entity'].lower())
    for row in winnersDF_films_D.iterrows():
        winners_years.append(row[1]['Year'])
        winners_entities.append(row[1]['Entity'].lower())
    for row in winnersDF_films_A.iterrows():
        winners_years.append(row[1]['Year'])
        winners_entities.append(row[1]['Entity'].lower())
    winners_film_data = {'Year': winners_years, 'Entity': winners_entities}

    # A DataFrame of the cleaned and lowercased winners is created.
    winnersDF_films = pd.DataFrame(data=winners_film_data)
    
    return winnersDF_films, winners_entities, winners_years

In [4]:
winnersDF_Film, entities, years = getWinnersDF_Film(allWinAndNom)
winnersDF_Film.head(3)

,Year,Entity
0,1927,wings
1,1928,the broadway melody
2,1929,all quiet on the western front


In [5]:
#! --- --- --- This creates a csv containing all of the titleBasics' movie titles lowercased
#! --- --- --- and as a new column. It takes a while so I'm leaving it commented out for now.

#print("titleBasics..")
#titleBasics = pd.read_csv("/Users/nicolespaar/Desktop/oscarDataAnalysis/Spaar/resources/data/title.basics.tsv", sep='\t', header=0, low_memory=False)
#lowercased_titles = []
#count = 0
#for row in titleBasics.iterrows():
#    try:
#        lower = row[1].primaryTitle.lower()
#        lowercased_titles.append(lower)
#    except:
#        count += 1
#        lowercased_titles.append("n/a")
#print(f"{count} movies had no 'primaryTitle'")
#titleBasics['lowercasedTitle'] = lowercased_titles
#titleBasics.to_csv("titleBasics_spaar.csv")

In [6]:
# IMdB title information .csv + my own munging of `title.lower()` is loaded in as $titleBasics.
# (SOURCE: https://www.imdb.com/interfaces/)
titleBasics = pd.read_csv("/Users/nicolespaar/Desktop/titleBasics_spaar.csv", index_col='tconst', low_memory=False)
titleBasics.head(3)

,Unnamed: 0,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,lowercasedTitle
tconst,,,,,,,,,,
tt0000001,0,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",carmencita
tt0000002,1,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",le clown et ses chiens
tt0000003,2,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",pauvre pierrot


In [7]:
#print("titleCrew..")
#titleCrew = pd.read_csv("/Users/nicolespaar/Desktop/title.crew.tsv", sep='\t', header=0)

#print("nameBasics..")
#nameBasics = pd.read_csv("/Users/nicolespaar/Desktop/name.basics.tsv", sep='\t', header=0, index_col='nconst')

#("titlePrincipals..")
#titlePrincipals = pd.read_csv("/Users/nicolespaar/Desktop/title.principals.tsv", sep='\t', header=0, index_col='nconst')
#titlePrincipals.head(3)

In [8]:
#!---- ---- ---- ---- From the Past ---- ---- ---- ----!#

def init_splinter():
    '''
    init_splinter() initializes a selenium chrome webdriver using a Tor proxy and returns it
    as a splinter browser wrapped object.
    '''
    executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
    browser = Browser('chrome', **executable_path, headless=False, incognito=True)
    return browser



def simmer_soup():
    '''
    simmer_soup() receives the browser object and returns the current page's parsed html as "Soup".
    '''
    html = browser.html
    soup = bs(html, 'html.parser')
    return html, soup

#!---- ---- ---- ---- From Scratch ---- ---- ---- ----!#

bechdel_base = "http://bechdeltest.com/api/v1/getMoviesByTitle?title="
bechdel_bools = []

def callAPI(movieToSearch):
    '''
    callAPI() calls the above url (Bechdel test API) with a movie title argument. The
    response is returned as a JSON in a variable ($conn). If the length of the response
    is less than or equal to 2, the response is a False (Either hasn't been Bechdel rated,
    or is rated a False for passing) and a 0 is returned. Else a True 1 is returned.
    '''
    conn = str(requests.get(bechdel_base + movieToSearch).json())
    if len(conn) <= 2:
        return 0
    else:
        return 1
    
    
     
def monthToNum(MonthName):
    '''
    monthtoNum() receives a Month name string and returns its corresponding int.
    '''
    return{
        'January': 1,
        'February': 2,
        'March': 3,
        'April': 4,
        'May': 5,
        'June': 6,
        'July': 7,
        'August': 8,
        'September': 9,
        'October': 10,
        'November': 11,
        'December': 12
    }[MonthName]



def getRateLimit():
    '''
    getRateLimit() returns a random number between 2.66 and 6.33, rounded to 2 decimals 
    places. The result is used as a rate-limiter for our scraping calls as to slow down
    requests and not be rude to IMdB.
    '''
    return round(random.uniform(2.66, 6.33), 2)

In [9]:
scraped = []

for row in winnersDF_Film.iterrows():
    
    # A random amount of time to throttle our requests is generated for each row iterration.
    throttle = getRateLimit()
    movieToSearch = row[1].Entity
    movieYear = row[1].Year
    
    # The downloaded IMdB database >> DataFrame is searched for the current iterration's movie.
    titleBasicsArray = titleBasics[titleBasics['lowercasedTitle'] == movieToSearch]
    titleBasicsRow_RAW = titleBasicsArray[titleBasicsArray['titleType'] == 'movie']
    titleBasicsRow = titleBasicsRow_RAW[titleBasicsRow_RAW['startYear'] == str(movieYear)]
    tconst = titleBasicsRow.index.to_list()
    tconst_str = ''.join(tconst)
    tconst_str = tconst_str[0:9]
    
    # If the length of the result of the search's tconst string is NOT 0 (empty/no result
    # found), then the associated details as well as the result of its Bechdel rating API
    # caller are appended as a list. That list is then appended into the $scraped array.
    if len(tconst_str) != 0:
        print(f"APPENDING info of: '{row[1].Entity}' ({row[1].Year}) with a throttle of: {throttle}s")
        temp = []
        temp.append(movieToSearch)
        temp.append(tconst_str) 
        temp.append(movieYear)
        time.sleep(throttle)
        bechdelBool = callAPI(movieToSearch)
        temp.append(bechdelBool)
        scraped.append(temp)  
        
    # If the length of the result of the search's tconst string IS 0 (empty/no results found),
    # then the needed details are scraped from IMdB's website.
    else:
        print(f"   [!] SCRAPING info of: '{row[1].Entity}' ({row[1].Year}) with a throttle of: {throttle}s")
    
        temp = []
        time.sleep(throttle)
        
        # The below URL can be used as an API of sorts by appending the $movieToSearch to it,
        # we must only replace the whitespace with a plus sign.
        base_url = "https://www.imdb.com/search/title/?title=" 
        movieToSearch = movieToSearch.replace(" ", "+")
        full_url = (base_url + movieToSearch)
        browser = init_splinter()
        browser.driver.minimize_window()
        browser.visit(full_url)
        
        # The results page is parsed with Beautifulsoup4 and a `soup` object returned. Then
        # the plus signs are replaced back with whitespace in our $movieToSearch. The `soup`
        # object is parsed and iterrated for the correct result that corresponds to the movie
        # we are after. Its <href> characters [7:16] are spliced as its `tconst` and appended
        # along with the rest of its associated information to a list. 
        html, soup = simmer_soup()
        movieToSearch = movieToSearch.replace("+", " ")
        results = soup.find_all('h3', class_='lister-item-header')
        for r in results:
            rYear = soup.select('h3 > span')[1].get_text(strip=True)
            rYear = rYear.replace("(", "")
            rYear = rYear.replace(")", "")
            if (r.a.text.lower() == movieToSearch) and movieToSearch not in temp:
                link_end = r.a['href']
                tconst_str = link_end[7:16]
                temp.append(movieToSearch)
                temp.append(tconst_str) 
                temp.append(movieYear)   
        browser.quit()
        
        #? The Bechdel rating for our new movie is acquired via the Bechdel callAPI() function,
        #? and appended to the rest of the information we scraped's list. Once complete, that
        #? list is then appended into the $scraped array.
        bechdelBool = callAPI(movieToSearch)
        temp.append(bechdelBool)
        scraped.append(temp)

print("\n\nHead of scrapes:")
print(scraped[0:5])
print("\nTail of scrapes:")
print(scraped[-6:-1])

APPENDING info of: 'wings' (1927) with a throttle of: 3.62s
   [!] SCRAPING info of: 'the broadway melody' (1928) with a throttle of: 3.54s
   [!] SCRAPING info of: 'all quiet on the western front' (1929) with a throttle of: 4.18s
   [!] SCRAPING info of: 'cimarron' (1930) with a throttle of: 4.44s
   [!] SCRAPING info of: 'grand hotel' (1931) with a throttle of: 4.87s
   [!] SCRAPING info of: 'cavalcade' (1932) with a throttle of: 5.54s
APPENDING info of: 'it happened one night' (1934) with a throttle of: 4.79s
APPENDING info of: 'mutiny on the bounty' (1935) with a throttle of: 5.78s
APPENDING info of: 'the great ziegfeld' (1936) with a throttle of: 4.07s
APPENDING info of: 'the life of emile zola' (1937) with a throttle of: 4.29s
APPENDING info of: 'you can't take it with you' (1938) with a throttle of: 3.72s
APPENDING info of: 'gone with the wind' (1939) with a throttle of: 2.79s
APPENDING info of: 'rebecca' (1940) with a throttle of: 2.73s
APPENDING info of: 'how green was my vall

In [10]:
# Check to see if TGF 2 && Cimarron are b0rked per usual:
print(scraped[45:48], "\n")
print(scraped[2:5])

[['the sting', 'tt0070735', 1973, 0], [0], ["one flew over the cuckoo's nest", 'tt0073486', 1975, 0]] 

[['all quiet on the western front', 'tt0020629', 1929, 1], ['cimarron', 'tt0053715', 1930, 1], ['grand hotel', 'tt7671068', 1931, 1]]


In [11]:
# Insert its data manually since it is.
scraped[46] = ['the godfather part ii', 'tt0071562', 1974, 0]
scraped[3] = ['cimarron', 'tt0021746', 1931, 1]

In [12]:
# Double check that it looks okay after manual insertion.
print(scraped[45:48], "\n")
print(scraped[2:5])

[['the sting', 'tt0070735', 1973, 0], ['the godfather part ii', 'tt0071562', 1974, 0], ["one flew over the cuckoo's nest", 'tt0073486', 1975, 0]] 

[['all quiet on the western front', 'tt0020629', 1929, 1], ['cimarron', 'tt0021746', 1931, 1], ['grand hotel', 'tt7671068', 1931, 1]]


In [13]:
Model_finalTitles = []
Model_finalTconsts = []
Model_finalYear = []
Model_finalBechdel = []

# The scraped data and Bechdel scores are appended to arrays, then they're made into a dict 
# that is used to create a DataFrame.
for winner in scraped:
    Model_finalTitles.append(winner[0])
    Model_finalTconsts.append(winner[1])
    Model_finalYear.append(winner[2])
    Model_finalBechdel.append(winner[3])

model_data = {'TCONST': Model_finalTconsts, 'TITLE': Model_finalTitles, 'YEAR': Model_finalYear, 'BECHDEL': Model_finalBechdel}    
modelDF = pd.DataFrame(data=model_data)
print("HEAD of [modelDF] results from `tconst` collection/scraping:\n")
print(modelDF.head())
print("\n\n\nTAIL of [modelDF] results from `tconst` collection/scraping:\n")
print(modelDF.tail())

HEAD of [modelDF] results from `tconst` collection/scraping:

      TCONST                           TITLE  YEAR  BECHDEL
0  tt0018578  wings                           1927  1      
1  tt0019729  the broadway melody             1928  0      
2  tt0020629  all quiet on the western front  1929  1      
3  tt0021746  cimarron                        1931  1      
4  tt7671068  grand hotel                     1931  1      



TAIL of [modelDF] results from `tconst` collection/scraping:

       TCONST                                            TITLE  YEAR  BECHDEL
85  tt2024544  12 years a slave                                 2013  1      
86  tt2562232  birdman or (the unexpected virtue of ignorance)  2014  0      
87  tt1895587  spotlight                                        2015  1      
88  tt4975722  moonlight                                        2016  1      
89  tt5580390  the shape of water                               2017  0      


In [14]:
def getModelDF_TITLE(count):
    return modelDF.loc[int(count - 1)].TITLE

def getModelDF_YEAR(count):
    return modelDF.loc[int(count - 1)].YEAR

def getModelDF_BECHDEL(count):
    return modelDF.loc[int(count - 1)].BECHDEL

tconstDoOvers = []
titleDoOvers = []
yearDoOvers = []
bechdelDoOvers = []
def changeTCONST_LIST(NEW_tconst, CHECK_title, CHECK_year, CHECK_bechdel):
    tconstDoOvers.append(NEW_tconst)
    titleDoOvers.append(CHECK_title)
    yearDoOvers.append(int(CHECK_year -1))
    bechdelDoOvers.append(CHECK_bechdel)
    
def clear():
    _ = call('clear' if os.name == 'posix' else 'cls')

In [15]:
# The below [empty] arrays declared are what we go and scrape for with the above created
# Dataframe of movies (modelDF).
scores = []
votecount = []
budgets = []
companies = []
releasedDays = []
releasedMonths = []
releasedYears = []
releasedCountries = []

imdb_base = "https://www.imdb.com/title/"

count = 0
iterLength = len(Model_finalTconsts)

for pk in Model_finalTconsts:
    
    count += 1
    CHECK_title = getModelDF_TITLE(count)
    CHECK_year = getModelDF_YEAR(count)
    CHECK_bechdel = getModelDF_BECHDEL(count)
    
    # A random amount of time to throttle our requests is generated for each `tconst` 
    # iterration. That `tconst` is concactenated onto the base IMdB movie page URL, and
    # it is then visited. That page's html is parsed with Beautifulsoup4 and a `soup`
    # object is returned.
    throttle = getRateLimit()
    imdb_end = pk
    url = (imdb_base + imdb_end)
    print(f"SCRAPE {count}/{iterLength}   >>>>   tconst: '{imdb_end}'   throttled by: {throttle}s   title: '{CHECK_title}'")
    time.sleep(throttle)
    browser = init_splinter()
    browser.visit(url)
    html, soup = simmer_soup()
    
    # The movie's ratings and vote count parent tag is extracted from the `soup` object and 
    # declared as $ratings.
    ratings_parent = soup.find('div', class_='ratingValue').strong['title']
    ratings = ratings_parent.split()
    
    # The movie's rating's score is appended to the current iterration's extraction list.
    score = ratings[0]
    scores.append(score)

    # The movie's rating's vote count is appended to the current iterration's extraction list.
    totalVotes = ratings[3]
    votecount.append(totalVotes.replace(",", ""))
    
    # Booleans for each of the troublesome features are declared as nil before attempting to
    # extract their associated categorical data from IMdB.
    budgetBOOL = False
    dayBOOL = False
    monthBOOL = False
    yearBOOL = False
    countryBOOL = False
    
    # The movie's certs parent tag is extracted from the `soup` object and declared as
    # $cert_raw. It is then iterrated through looking for <h4> tags' text that equal
    # `Budget`, `Production Co`, and `Release Date` to extract as attributes to append
    # as associated information/features of the dataset.
    cert_raw = soup.find_all('div', class_='txt-block')
    for cert in cert_raw:
        try:
            
            # If the `Budget` <h4> text is encountered within the iterration of the <div>s of
            # .txt-block, and if its length is not null, its Boolean is set to true and its
            # value from the 8th character on are appended to the current iterration's 
            # extraction list.
            if cert.h4.text == "Budget:":
                budget_RAW = cert.text.split()
                budget = budget_RAW[0]               
                if len(budget[8:]) != 0:
                    budgetBOOL = True
                    budgets.append(budget[8:].replace(",", ""))

            # If the `Production Co:` <h4> text is encountered within the iterration of the 
            # <div>s of .txt-block, its Boolean is set to true and its value from the 3rd 
            # indice/element on are appended to the current iterration's extraction list.
            elif cert.h4.text == "Production Co:":
                companies_RAW = cert.text.split()
                company = companies_RAW[2]            
                companies.append(company)   

            # If the `Production Co:` <h4> text is encountered within the iterration of the 
            # <div>s of .txt-block, its 3rd, 4th, and 5th indices/elements are extracted out
            # as new $VARs. The month ($release2) is converted to a numerical representation
            # of its string value, too.
            elif cert.h4.text == "Release Date:":
                release_RAW = cert.text.split()
                release1 = release_RAW[2]
                release2 = release_RAW[3]
                release3 = release_RAW[4]
                release2 = monthToNum(release2)
             
                # If the day ($release1) is not null, its Boolean is set to true and its value
                # is appended to the current iterration's extraction list.
                if len(release1) != 0:
                    dayBOOL = True
                    releasedDays.append(release1)
                
                # If the month ($release2) is not 1-12, its Boolean is set to true and its 
                # value is appended to the current iterration's extraction list.
                if release2 in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]:
                    monthBOOL = True
                    releasedMonths.append(release2)
                
                # If the year ($release3) is not null, its Boolean is set to true and its 
                # value is appended to the current iterration's extraction list.
                if len(release3) != 0:
                    yearBOOL = True
                    releasedYears.append(release3)
                
                # The 6th indice/element of $release_RAW is extracted out as a new variable,
                # $release_country. It also has its parens removed from its contents.
                release_country = release_RAW[5]
                release_country = release_country.replace("(", "")
                release_country = release_country.replace(")", "")
                
                # If the country ($release_country) is not null, its Boolean is set to true
                # and its value is appended to the current iterration's extraction list.
                if len(release_country) != 0:
                    countryBOOL = True
                    releasedCountries.append(release_country)
                    
        # Each iterration errors out at some point, so this allows the program to continue
        # regardless of what errors are being thrown about it.
        except:
            pass
    
    # I tried many ways to make this more elegant of code, but had to cut my losses for this
    # version that, while ugly: does work as intended. This is a refactored version of what
    # was all individual $___BOOL checks. If any $___BOOL $VARs are not True, then the user
    # is prompted if the existing `tconst` is indeed correct.
    if budgetBOOL != True or dayBOOL!= True or monthBOOL != True or yearBOOL != True or countryBOOL != True:
        print(f"   [!] ERROR [!]")
        tconstCheck = None
        while tconstCheck not in ("y", "n"):
            tconstCheck = str(input(f"   [!] Is {pk} the correct TCONST for '{CHECK_title}' ({CHECK_year})? [!]    [y]/[n] "))
            
            # If YES (the existing `tconst` is correct), the user is prompted to manually
            # input the data that was unable to be scraped.
            if tconstCheck == "y":
                if budgetBOOL != True:
                    manualInsertion = float(input(f"   [!] ENTER {pk}'s   $BUDGET$   (As a whole number): [!] "))
                    budgets.append(manualInsertion)
                if dayBOOL != True:
                    manualInsertion = float(input(f"   [!] ENTER {pk}'s   $RELEASE DAY$   (As a whole number, no leading zeros): [!] "))
                    releasedDays.append(manualInsertion)
                if monthBOOL != True:
                    manualInsertion = float(input(f"   [!] ENTER {pk}'s   $RELEASE_MONTH$   (As a whole number, no leading zeros): [!] "))
                    releasedMonths.append(manualInsertion)
                if yearBOOL != True:
                    manualInsertion = float(input(f"   [!] ENTER {pk}'s   $RELEASE_YEAR$   (As a whole number, 4 digits): [!] "))
                    releasedYears.append(manualInsertion)
                if countryBOOL != True:
                    manualInsertion = str(input(f"   [!] ENTER {pk}'s   $RELEASE_COUNTRY$   (As an uppercase abbr if +1 word eg: 'USA', else as a Titlecase string like 'Japan'): [!] "))
                    releasedCountries.append(manualInsertion)
            
            # If NO (the existing `tconst` is incorrect), the user is prompted to manually
            # input the correct `tconst`, and if it has not already been appended to the
            # $tconstDoOvers, then it is added. 
            elif tconstCheck == "n":
                NEW_tconst = str(input("   [!] Enter the correct TCONST: [!] "))         
                if NEW_tconst not in tconstDoOvers:
                    changeTCONST_LIST(NEW_tconst, CHECK_title, CHECK_year, CHECK_bechdel)
                    
                # Also, each bool that was left False (the scraper couldn't locate the correct
                # data) signals a value of `None` to be appended to the current iterration's
                # extraction list so that its row can later be dropped in favor of $tconstDoOvers    
                if budgetBOOL != True:
                    budgets.append(None)
                if dayBOOL != True:
                    releasedDays.append(None)
                if monthBOOL != True:
                    releasedMonths.append(None)
                if yearBOOL != True:
                    releasedYears.append(None)
                if countryBOOL != True:
                    releasedCountries.append(None)
            else:
                clear()
                print("      [!] [!]    You must enter 'y' or 'n'    [!] [!]")
                time.sleep(4)
                                    
    
    browser.quit()
    
os.system('say "skeet skeet"')

SCRAPE 1/90   >>>>   tconst: 'tt0018578'   throttled by: 3.72s   title: 'wings'
SCRAPE 2/90   >>>>   tconst: 'tt0019729'   throttled by: 2.99s   title: 'the broadway melody'
SCRAPE 3/90   >>>>   tconst: 'tt0020629'   throttled by: 6.21s   title: 'all quiet on the western front'
SCRAPE 4/90   >>>>   tconst: 'tt0021746'   throttled by: 4.37s   title: 'cimarron'
SCRAPE 5/90   >>>>   tconst: 'tt7671068'   throttled by: 5.24s   title: 'grand hotel'
   [!] ERROR [!]
   [!] Is tt7671068 the correct TCONST for 'grand hotel' (1931)? [!]    [y]/[n] n
   [!] Enter the correct TCONST: [!] tt0022958
SCRAPE 6/90   >>>>   tconst: 'tt0023876'   throttled by: 4.4s   title: 'cavalcade'
SCRAPE 7/90   >>>>   tconst: 'tt0025316'   throttled by: 4.3s   title: 'it happened one night'
SCRAPE 8/90   >>>>   tconst: 'tt0026752'   throttled by: 3.16s   title: 'mutiny on the bounty'
   [!] ERROR [!]
   [!] Is tt0026752 the correct TCONST for 'mutiny on the bounty' (1935)? [!]    [y]/[n] y
   [!] ENTER tt0026752's 

SCRAPE 55/90   >>>>   tconst: 'tt0083987'   throttled by: 3.17s   title: 'gandhi'
SCRAPE 56/90   >>>>   tconst: 'tt0086425'   throttled by: 3.9s   title: 'terms of endearment'
SCRAPE 57/90   >>>>   tconst: 'tt0086879'   throttled by: 5.96s   title: 'amadeus'
SCRAPE 58/90   >>>>   tconst: 'tt0089755'   throttled by: 2.82s   title: 'out of africa'
SCRAPE 59/90   >>>>   tconst: 'tt0091763'   throttled by: 4.85s   title: 'platoon'
SCRAPE 60/90   >>>>   tconst: 'tt0093389'   throttled by: 4.13s   title: 'the last emperor'
SCRAPE 61/90   >>>>   tconst: 'tt0095953'   throttled by: 4.99s   title: 'rain man'
SCRAPE 62/90   >>>>   tconst: 'tt0097239'   throttled by: 4.55s   title: 'driving miss daisy'
SCRAPE 63/90   >>>>   tconst: 'tt0099348'   throttled by: 3.17s   title: 'dances with wolves'
SCRAPE 64/90   >>>>   tconst: 'tt0102926'   throttled by: 2.93s   title: 'the silence of the lambs'
SCRAPE 65/90   >>>>   tconst: 'tt0105695'   throttled by: 4.39s   title: 'unforgiven'
SCRAPE 66/90   >>>>

In [16]:
modelDF['RATING'] = scores
modelDF['VOTES'] = votecount
modelDF['PRODUCTION_CO'] = companies

modelDF['BUDGET'] = budgets
modelDF['RELEASE_DAY'] = releasedDays
modelDF['RELEASE_MON'] = releasedMonths
modelDF['RELEASE_YEAR'] = releasedYears
modelDF['COUNTRY'] = releasedCountries

modelDF.head()

,TCONST,TITLE,YEAR,BECHDEL,RATING,VOTES,PRODUCTION_CO,BUDGET,RELEASE_DAY,RELEASE_MON,RELEASE_YEAR,COUNTRY
0,tt0018578,wings,1927,1,7.7,10234,Paramount,2000000,5,1.0,1929,USA
1,tt0019729,the broadway melody,1928,0,6.1,5836,Metro-Goldwyn-Mayer,379000,6,6.0,1929,USA
2,tt0020629,all quiet on the western front,1929,1,8.0,53729,Universal,1448864,24,8.0,1930,USA
3,tt0021746,cimarron,1931,1,5.9,4837,RKO,1433000,9,2.0,1931,USA
4,tt7671068,grand hotel,1931,1,6.6,1128,Unbelievable,None,17,6.0,2019,USA


In [17]:
# The below [empty] arrays declared are what we go and scrape for with the $tconstDoOvers
# (they originally had the incorrect `tconst`.
scores = []
votecount = []
budgets = []
companies = []
releasedDays = []
releasedMonths = []
releasedYears = []
releasedCountries = []

imdb_base = "https://www.imdb.com/title/"

count = 0
iterLength = len(tconstDoOvers)

for pk in tconstDoOvers:
    
    count += 1
    
    # A random amount of time to throttle our requests is generated for each `tconst` 
    # iterration. That `tconst` is concactenated onto the base IMdB movie page URL, and
    # it is then visited. That page's html is parsed with Beautifulsoup4 and a `soup`
    # object is returned.
    throttle = getRateLimit()
    imdb_end = pk
    url = (imdb_base + imdb_end)
    print(f"SCRAPE {count}/{iterLength}   >>>>   tconst: '{imdb_end}'   throttled by: {throttle}s")   #title: '{CHECK_title}'")
    time.sleep(throttle)
    browser = init_splinter()
    browser.visit(url)
    html, soup = simmer_soup()
    
    # The movie's ratings and vote count parent tag is extracted from the `soup` object and 
    # declared as $ratings.
    ratings_parent = soup.find('div', class_='ratingValue').strong['title']
    ratings = ratings_parent.split()
    
    # The movie's rating's score is appended to the current iterration's extraction list.
    score = ratings[0]
    scores.append(score)

    # The movie's rating's vote count is appended to the current iterration's extraction list.
    totalVotes = ratings[3]
    votecount.append(totalVotes.replace(",", ""))
    
    # Booleans for each of the troublesome features are declared as nil before attempting to
    # extract their associated categorical data from IMdB.
    budgetBOOL = False
    dayBOOL = False
    monthBOOL = False
    yearBOOL = False
    countryBOOL = False
    
    # The movie's certs parent tag is extracted from the `soup` object and declared as
    # $cert_raw. It is then iterrated through looking for <h4> tags' text that equal
    # `Budget`, `Production Co`, and `Release Date` to extract as attributes to append
    # as associated information/features of the dataset.
    cert_raw = soup.find_all('div', class_='txt-block')
    for cert in cert_raw:
        try:
            
            # If the `Budget` <h4> text is encountered within the iterration of the <div>s of
            # .txt-block, and if its length is not null, its Boolean is set to true and its
            # value from the 8th character on are appended to the current iterration's 
            # extraction list.
            if cert.h4.text == "Budget:":
                budget_RAW = cert.text.split()
                budget = budget_RAW[0]               
                if len(budget[8:]) != 0:
                    budgetBOOL = True
                    budgets.append(budget[8:].replace(",", ""))

            # If the `Production Co:` <h4> text is encountered within the iterration of the 
            # <div>s of .txt-block, its Boolean is set to true and its value from the 3rd 
            # indice/element on are appended to the current iterration's extraction list.
            elif cert.h4.text == "Production Co:":
                companies_RAW = cert.text.split()
                company = companies_RAW[2]            
                companies.append(company)   

            # If the `Production Co:` <h4> text is encountered within the iterration of the 
            # <div>s of .txt-block, its 3rd, 4th, and 5th indices/elements are extracted out
            # as new $VARs. The month ($release2) is converted to a numerical representation
            # of its string value, too.
            elif cert.h4.text == "Release Date:":
                release_RAW = cert.text.split()
                release1 = release_RAW[2]
                release2 = release_RAW[3]
                release3 = release_RAW[4]
                release2 = monthToNum(release2)
             
                # If the day ($release1) is not null, its Boolean is set to true and its value
                # is appended to the current iterration's extraction list.
                if len(release1) != 0:
                    dayBOOL = True
                    releasedDays.append(release1)
                
                # If the month ($release2) is not 1-12, its Boolean is set to true and its 
                # value is appended to the current iterration's extraction list.
                if release2 in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]:
                    monthBOOL = True
                    releasedMonths.append(release2)
                
                # If the year ($release3) is not null, its Boolean is set to true and its 
                # value is appended to the current iterration's extraction list.
                if len(release3) != 0:
                    yearBOOL = True
                    releasedYears.append(release3)
                
                # The 6th indice/element of $release_RAW is extracted out as a new variable,
                # $release_country. It also has its parens removed from its contents.
                release_country = release_RAW[5]
                release_country = release_country.replace("(", "")
                release_country = release_country.replace(")", "")
                
                # If the country ($release_country) is not null, its Boolean is set to true
                # and its value is appended to the current iterration's extraction list.
                if len(release_country) != 0:
                    countryBOOL = True
                    releasedCountries.append(release_country)
                    
        # Each iterration errors out at some point, so this allows the program to continue
        # regardless of what errors are being thrown about it.
        except:
            pass
    
    # I tried many ways to make this more elegant of code, but had to cut my losses for this
    # version that, while ugly: does work as intended. This is a refactored version of what
    # was all individual $___BOOL checks. If any $___BOOL $VARs are not True, then the user
    # is prompted if the existing `tconst` is indeed correct.
    if budgetBOOL != True or dayBOOL!= True or monthBOOL != True or yearBOOL != True or countryBOOL != True:
        print(f"   [!] ERROR [!]")
        tconstCheck = None
        while tconstCheck not in ("y", "n"):
            tconstCheck = str(input(f"   [!] Is {pk} the correct TCONST for '{CHECK_title}' ({CHECK_year})? [!]    [y]/[n] "))
            
            # If YES (the existing `tconst` is correct), the user is prompted to manually
            # input the data that was unable to be scraped.
            if tconstCheck == "y":
                if budgetBOOL != True:
                    manualInsertion = float(input(f"   [!] ENTER {pk}'s   $BUDGET$   MANUALLY (As a whole number): [!] "))
                    budgets.append(manualInsertion)
                if dayBOOL != True:
                    manualInsertion = float(input(f"   [!] ENTER {pk}'s   $RELEASE DAY$   MANUALLY (As a whole number, no leading zeros): [!] "))
                    releasedDays.append(manualInsertion)
                if monthBOOL != True:
                    manualInsertion = float(input(f"   [!] ENTER {pk}'s   $RELEASE_MONTH$   MANUALLY (As a whole number, no leading zero): [!] "))
                    releasedMonths.append(manualInsertion)
                if yearBOOL != True:
                    manualInsertion = float(input(f"   [!] ENTER {pk}'s   $RELEASE_YEAR$   MANUALLY (As a whole number, 4 digits): [!] "))
                    releasedYears.append(manualInsertion)
                if countryBOOL != True:
                    manualInsertion = str(input(f"   [!] ENTER {pk}'s   $RELEASE_COUNTRY$   MANUALLY (As a string abbreviation, eg: 'USA'): [!] "))
                    releasedCountries.append(manualInsertion)
            
            # If NO (the existing `tconst` is incorrect), the user is prompted to manually
            # input the correct `tconst`, and if it has not already been appended to the
            # $tconstDoOvers, then it is added. 
            elif tconstCheck == "n":
                NEW_tconst = str(input("   [!] Enter the correct TCONST: [!] "))         
                if NEW_tconst not in tconstDoOvers:
                    changeTCONST_LIST(NEW_tconst)
                    
                # Also, each bool that was left False (the scraper couldn't locate the correct
                # data) signals a value of `None` to be appended to the current iterration's
                # extraction list so that its row can later be dropped in favor of $tconstDoOvers    
                if budgetBOOL != True:
                    budgets.append(None)
                if dayBOOL != True:
                    releasedDays.append(None)
                if monthBOOL != True:
                    releasedMonths.append(None)
                if yearBOOL != True:
                    releasedYears.append(None)
                if countryBOOL != True:
                    releasedCountries.append(None)
            else:
                clear()
                print("      [!] [!]    You must enter 'y' or 'n'    [!] [!]")
                time.sleep(4)
                                    
    
    browser.quit()

os.system('say "skeet skeet"')

SCRAPE 1/1   >>>>   tconst: 'tt0022958'   throttled by: 5.62s


In [18]:
# Peep at the DataFrame created from received information from previous (main) big iterrator.
redoData = {'TCONST': tconstDoOvers, 'TITLE': titleDoOvers, 'YEAR': yearDoOvers, 'BECHDEL': bechdelDoOvers}
redoDF = pd.DataFrame(data=redoData)
redoDF

,TCONST,TITLE,YEAR,BECHDEL
0,tt0022958,grand hotel,1931,1


In [19]:
# Add in the new data scraped from IMdB now with the correct `tconst` to the $redoDF.
redoDF['RATING'] = scores
redoDF['VOTES'] = votecount
redoDF['PRODUCTION_CO'] = companies

redoDF['BUDGET'] = budgets
redoDF['RELEASE_DAY'] = releasedDays
redoDF['RELEASE_MON'] = releasedMonths
redoDF['RELEASE_YEAR'] = releasedYears
redoDF['COUNTRY'] = releasedCountries

redoDF

,TCONST,TITLE,YEAR,BECHDEL,RATING,VOTES,PRODUCTION_CO,BUDGET,RELEASE_DAY,RELEASE_MON,RELEASE_YEAR,COUNTRY
0,tt0022958,grand hotel,1931,1,7.5,15822,Metro-Goldwyn-Mayer,700000,11,9,1932,USA


In [20]:
# All rows containing our appended value of $None (the rows that had an incorrect `tconst`)
# are dropped from the $modelDF dataframe.
modelDF.dropna(inplace=True)
modelDF

,TCONST,TITLE,YEAR,BECHDEL,RATING,VOTES,PRODUCTION_CO,BUDGET,RELEASE_DAY,RELEASE_MON,RELEASE_YEAR,COUNTRY
0,tt0018578,wings,1927,1,7.7,10234,Paramount,2000000,5,1.0,1929,USA
1,tt0019729,the broadway melody,1928,0,6.1,5836,Metro-Goldwyn-Mayer,379000,6,6.0,1929,USA
2,tt0020629,all quiet on the western front,1929,1,8.0,53729,Universal,1448864,24,8.0,1930,USA
3,tt0021746,cimarron,1931,1,5.9,4837,RKO,1433000,9,2.0,1931,USA
5,tt0023876,cavalcade,1932,1,6.0,4018,Fox,1180280,15,4.0,1933,USA
6,tt0025316,it happened one night,1934,1,8.1,85490,Columbia,325000,22,2.0,1934,USA
7,tt0026752,mutiny on the bounty,1935,1,7.8,19073,Metro-Goldwyn-Mayer,1950000,22,11.0,1935,USA
8,tt0027698,the great ziegfeld,1936,0,6.8,6469,Metro-Goldwyn-Mayer,2.183e+06,8,4.0,1936,USA
9,tt0029146,the life of emile zola,1937,0,7.3,6328,Warner,700000,2,10.0,1937,USA
10,tt0030993,you can't take it with you,1938,0,8.0,21379,Columbia,1644736,29,9.0,1938,USA


In [21]:
# Both Dataframes are joined upon each other for the final encompassing dataset consisting of 
# $modelDF and $redoDF. We can now run a model through it.
allDF = modelDF.append(redoDF)
allDF

,TCONST,TITLE,YEAR,BECHDEL,RATING,VOTES,PRODUCTION_CO,BUDGET,RELEASE_DAY,RELEASE_MON,RELEASE_YEAR,COUNTRY
0,tt0018578,wings,1927,1,7.7,10234,Paramount,2000000,5,1.0,1929,USA
1,tt0019729,the broadway melody,1928,0,6.1,5836,Metro-Goldwyn-Mayer,379000,6,6.0,1929,USA
2,tt0020629,all quiet on the western front,1929,1,8.0,53729,Universal,1448864,24,8.0,1930,USA
3,tt0021746,cimarron,1931,1,5.9,4837,RKO,1433000,9,2.0,1931,USA
5,tt0023876,cavalcade,1932,1,6.0,4018,Fox,1180280,15,4.0,1933,USA
6,tt0025316,it happened one night,1934,1,8.1,85490,Columbia,325000,22,2.0,1934,USA
7,tt0026752,mutiny on the bounty,1935,1,7.8,19073,Metro-Goldwyn-Mayer,1950000,22,11.0,1935,USA
8,tt0027698,the great ziegfeld,1936,0,6.8,6469,Metro-Goldwyn-Mayer,2.183e+06,8,4.0,1936,USA
9,tt0029146,the life of emile zola,1937,0,7.3,6328,Warner,700000,2,10.0,1937,USA
10,tt0030993,you can't take it with you,1938,0,8.0,21379,Columbia,1644736,29,9.0,1938,USA


In [22]:
# The $allDF DataFrame is ordered by the values in the `YEAR` column
allDF = allDF.sort_values(by = 'YEAR')
allDF

,TCONST,TITLE,YEAR,BECHDEL,RATING,VOTES,PRODUCTION_CO,BUDGET,RELEASE_DAY,RELEASE_MON,RELEASE_YEAR,COUNTRY
0,tt0018578,wings,1927,1,7.7,10234,Paramount,2000000,5,1.0,1929,USA
1,tt0019729,the broadway melody,1928,0,6.1,5836,Metro-Goldwyn-Mayer,379000,6,6.0,1929,USA
2,tt0020629,all quiet on the western front,1929,1,8.0,53729,Universal,1448864,24,8.0,1930,USA
0,tt0022958,grand hotel,1931,1,7.5,15822,Metro-Goldwyn-Mayer,700000,11,9.0,1932,USA
3,tt0021746,cimarron,1931,1,5.9,4837,RKO,1433000,9,2.0,1931,USA
5,tt0023876,cavalcade,1932,1,6.0,4018,Fox,1180280,15,4.0,1933,USA
6,tt0025316,it happened one night,1934,1,8.1,85490,Columbia,325000,22,2.0,1934,USA
7,tt0026752,mutiny on the bounty,1935,1,7.8,19073,Metro-Goldwyn-Mayer,1950000,22,11.0,1935,USA
8,tt0027698,the great ziegfeld,1936,0,6.8,6469,Metro-Goldwyn-Mayer,2.183e+06,8,4.0,1936,USA
9,tt0029146,the life of emile zola,1937,0,7.3,6328,Warner,700000,2,10.0,1937,USA
